In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sklearn.datasets as sk_data
import sklearn.model_selection as sk_selection
import sklearn.metrics as sk_metrics
import sklearn.ensemble as sk_ensemble
import random_neural_net_models.tabular as rnnm_tab
import torch.optim as optim
import random_neural_net_models.losses as rnnm_loss
import random_neural_net_models.learner as rnnm_learner
import random_neural_net_models.data as rnnm_data
import random_neural_net_models.utils as rnnm_utils
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
rnnm_utils.make_deterministic(42)

In [ ]:
device = rnnm_utils.get_device()
device

In [ ]:
X, Y = sk_data.load_linnerud(return_X_y=True)

In [ ]:
X

In [ ]:
y_chin, y_situp, y_jump = Y.T

In [ ]:
y_chin

In [ ]:
X0, X1, y0, y1 = sk_selection.train_test_split(
    X, y_chin, test_size=0.2, random_state=42
)

In [ ]:
y0.min(), y0.max()

In [ ]:
import seaborn as sns

sns.histplot(x=y0)

## baseline model

In [ ]:
base_model = sk_ensemble.HistGradientBoostingRegressor()

In [ ]:
base_model.fit(X0, y0)

In [ ]:
y_pred_base = base_model.predict(X1)

In [ ]:
import math

rmse_base = math.sqrt(
    sk_metrics.mean_squared_error(y_true=y1, y_pred=y_pred_base)
)
rmse_base

## neural net model

In [ ]:
ds_train = rnnm_data.NumpyTrainingDataset(X0, y0)
ds_valid = rnnm_data.NumpyTrainingDataset(X1, y1)

In [ ]:
ds_train[0]

In [ ]:
len(X0)

In [ ]:
len(X0) / 100

In [ ]:
from torch.utils.data import RandomSampler
import torch

In [ ]:
batch_size = 100

sampler = RandomSampler(
    ds_train,
    replacement=True,
    num_samples=int(1e5),
    generator=torch.manual_seed(42),
)
dl_train = DataLoader(
    ds_train,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock,
    sampler=sampler,
    drop_last=True,
)
dl_valid = DataLoader(
    ds_valid,
    batch_size=batch_size,
    collate_fn=rnnm_data.collate_numpy_dataset_to_xyblock,
)

In [ ]:
next(iter(dl_train))

In [ ]:
import numpy as np

np.isfinite(X0).all(axis=0)

In [ ]:
n_hidden = [200, 100]
do_impute = False
impute_bias_source = rnnm_tab.BiasSources.zero
n_features = X0.shape[1]
n_classes = len(set(y0))

mean, std = y0.mean(), y0.std()

model = rnnm_tab.TabularModelRegression(
    n_features=n_features,
    n_hidden=n_hidden,
    mean=mean,
    std=std,
    use_batch_norm=True,
)

In [ ]:
learning_rate = 0.1
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss = rnnm_loss.MSELossXy()
loss_callback = rnnm_learner.TrainLossCallback()

save_dir = Path("./models")

callbacks = [loss_callback]

In [ ]:
learner = rnnm_learner.Learner(
    model,
    optimizer,
    loss,
    callbacks=callbacks,
    save_dir=save_dir,
    device=device,
    show_epoch_progress=True,
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

learner.find_learning_rate(
    dl_train, n_epochs=10, lr_find_callback=lr_find_callback
)

In [ ]:
lr_find_callback.get_losses()

In [ ]:
lr_find_callback.plot(yscale="log", ylim=(1e2, 1e4))

In [ ]:
learning_rate = 1e-3
n_epochs = 2

scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=learning_rate,
    epochs=n_epochs,
    steps_per_epoch=len(dl_train),
)
scheduler_callback = rnnm_learner.EveryBatchSchedulerCallback(scheduler)
learner.update_callback(scheduler_callback)

In [ ]:
learner.fit(dl_train, n_epochs=n_epochs, dataloader_valid=dl_valid)

In [ ]:
loss_callback.plot(yscale="log")

In [ ]:
y_pred_nn = learner.predict(dl_valid).detach().numpy().ravel()
y_pred_nn[:3]

In [ ]:
sns.histplot(x=y_pred_base)

In [ ]:
sns.histplot(x=y_pred_nn)

In [ ]:
rmse_nn = math.sqrt(sk_metrics.mean_squared_error(y_true=y1, y_pred=y_pred_nn))
rmse_nn, rmse_base